In [ ]:
import torch

model = torch.load('exported_models/head_1_1_1_1.pt')
model.eval()
input_tensor = torch.rand(1, 1024, 20, 20)
with torch.no_grad():
    out = model(input_tensor)
torch.onnx.export(model, input_tensor, 'test.onnx')


In [ ]:
import os

from torch2tflite.converter import Torch2TFLiteConverter

def arch2tflite(name: str, out_dir: str):

    base_path = os.path.join(out_dir, name)
    
    if 'backbone' in name:
        dummy_input_shape = (1, 3, 640, 640)
    else:
        dummy_input_shape = (1, 1024, 20, 20)

    conv = Torch2TFLiteConverter(
        base_path + '.pt',
        base_path + '.tflite',
        target_shape = dummy_input_shape,
    )
    conv.convert(remove_tmp_dir=True)
    

from tqdm.auto import tqdm

out_dir = 'exported_models'

file_list = [d for d in os.listdir(out_dir) if d.endswith('pt') and 'head' in d]

for torch_file in tqdm(file_list):
    arch2tflite(torch_file[:-3], out_dir)
    os.remove(os.path.join(out_dir, torch_file))